### 什么是Seq2Seq？

Seq2Seq (Sequence-to-Sequence) 是一种深度学习模型，它的作用是将一个序列（按顺序排列的数据）转换成另一个序列。 

**您可以把它想象成一个翻译器，它能够理解输入数据的顺序，并将其转换成另一种顺序的输出数据。**

**以下是一些 Seq2Seq 应用的例子：**

* **机器翻译：**  将中文翻译成英文，或者将英文翻译成日文。
* **语音识别：** 将语音转换成文字。
* **文本摘要：** 将一篇长文章转换成简短的摘要。
* **聊天机器人：**  根据用户的问题，生成相应的回答。


Seq2Seq 模型通常由两个主要部分组成：**编码器** 和 **解码器**。

* **编码器：** 负责读取输入序列，并将其压缩成一个固定长度的向量（称为上下文向量）。这个向量包含了输入序列的关键信息。
* **解码器：** 负责读取上下文向量，并根据其中的信息生成新的序列。


**Seq2Seq 模型的优点：**

* **能够处理长度不同的输入和输出序列。** 例如，输入一个长句子，输出一个短语。
* **能够考虑上下文信息。**  编码器将整个输入序列的信息压缩到上下文向量中，解码器利用这些信息生成更准确的输出。

Seq2Seq 模型在自然语言处理、语音识别等领域有着广泛的应用，并且还在不断发展和完善中。

### 为什么需要Seq2Seq？

我们需要生成一个新的队列（序列）主要是因为以下几个原因：

1. **实现序列到序列的转换:** Seq2Seq 模型的目标是将一个序列转换成另一个序列，例如将英语翻译成法语。原始序列和目标序列通常是不同的，因此需要生成一个新的序列来表示目标序列。

2. **满足特定任务的需求:** 不同的任务对输出序列的要求不同。例如，在机器翻译中，我们需要生成一个语法正确、语义流畅的目标语言序列；在文本摘要中，我们需要生成一个简洁、概括原文主要内容的序列。

3. **适应不同的应用场景:** Seq2Seq 模型可以应用于各种不同的场景，例如机器翻译、语音识别、文本摘要、对话生成等。每个应用场景都需要生成不同类型的序列。

### 实现步骤

1. **数据准备**
   - 将对话数据准备成输入-输出对 (input-output pairs)，比如：输入为「おはようございます。」，输出为「今日はとても良いお天気ですね。」。
   - 使用日语分词器（例如 `cl-tohoku/bert-base-japanese`）进行分词，将句子转为词 ID 序列。

2. **模型结构**
   - **编码器 (Encoder)**：使用 LSTM 或 GRU，将输入句子转换为隐藏状态表示。
   - **解码器 (Decoder)**：另一个 LSTM 或 GRU，将编码器的隐藏状态作为初始输入，以逐步生成输出句子。
   - **Attention 机制**（可选）：可以使用 Attention 机制，使解码器更好地关注输入序列中重要的信息。

3. **训练过程**
   - 定义损失函数（通常是交叉熵损失），并让模型学习从输入句子预测输出句子。
   - 将数据分批次输入模型进行训练，调优模型参数。

4. **推理（Inference）**
   - 在完成训练后，输入一个句子到编码器，通过解码器生成预测句子。
   - 使用 beam search 或者 greedy decoding 从模型中获取完整句子。

### 解释各部分的功能：

<center>
<img src="res/seq2seq.png" alt="Image description" style="width:60%; height:60%;"/>
</center>

- **Encoder**：将输入序列转换为嵌入表示并通过 LSTM 获取最后一个时间步的隐藏状态 `hidden` 和细胞状态 `cell`，作为编码后的句子表示，传递给解码器。

- **Decoder**：在每个时间步接受上一个时间步的输出词，更新隐藏状态并输出预测的下一个词。通过 `fc_out` 层将 LSTM 的输出映射到目标词汇表大小。

- **Seq2Seq**：
  - 调用 `encoder` 对输入序列进行编码。
  - 用编码的隐藏状态和细胞状态初始化 `decoder`。
  - 在每个时间步中，按 `teacher_forcing_ratio` 决定下一个输入是目标词还是预测词。
  - 将每一步的预测结果存储在 `outputs` 中，以便于计算损失或生成最终输出。


这样可以通过训练让模型学习到对话补全的规律。尽管 LSTM 或 GRU 相比 Transformer 模型效果可能弱一些，但它更易于训练和部署，能满足一些简单对话生成的需求。

### 实现过程

现在我们来实现一个Seq2Seq的模型。

#### 1. 引入需要的包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#### 2. Encoder的定义

In [2]:
# 定义编码器 (Encoder) 模型
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        """
        初始化编码器。
        参数:
            input_dim (int): 输入词汇表大小（即输入词的种类数）
            emb_dim (int): 嵌入层维度（每个词的嵌入向量大小）
            hid_dim (int): 隐藏层维度（LSTM的隐藏状态大小）
        """
        super(Encoder, self).__init__()
        # 定义嵌入层，将每个输入词转为稠密向量
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # 定义LSTM层，将嵌入向量传入LSTM以生成隐藏状态
        self.rnn = nn.LSTM(emb_dim, hid_dim)

    def forward(self, src):
        """
        编码器的前向传播。
        参数:
            src (Tensor): 输入张量，形状为 (src_len, batch_size)
        返回:
            hidden (Tensor): 最后一个时间步的隐藏状态，形状为 (1, batch_size, hid_dim)
            cell (Tensor): 最后一个时间步的细胞状态，形状为 (1, batch_size, hid_dim)
        """
        # 将输入词索引序列转换为嵌入向量
        embedded = self.embedding(src)  # embedded: (src_len, batch_size, emb_dim)
        # 通过LSTM层，获取隐藏状态和细胞状态
        outputs, (hidden, cell) = self.rnn(embedded)
        # 只返回最后一个时间步的隐藏状态和细胞状态
        return hidden, cell

#### 3. Decoder的定义

In [3]:

# 定义解码器 (Decoder) 模型
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim):
        """
        初始化解码器。
        参数:
            output_dim (int): 输出词汇表大小（即输出词的种类数）
            emb_dim (int): 嵌入层维度
            hid_dim (int): 隐藏层维度
        """
        super(Decoder, self).__init__()
        # 嵌入层，将每个输出词转为稠密向量
        self.embedding = nn.Embedding(output_dim, emb_dim)
        # LSTM层，处理嵌入向量并生成新的隐藏状态
        self.rnn = nn.LSTM(emb_dim, hid_dim)
        # 全连接层，用于将隐藏状态转为词汇表大小的向量
        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden, cell):
        """
        解码器的前向传播。
        参数:
            input (Tensor): 当前时间步的输入张量（目标词），形状为 (batch_size)
            hidden (Tensor): 上一个时间步的隐藏状态，形状为 (1, batch_size, hid_dim)
            cell (Tensor): 上一个时间步的细胞状态，形状为 (1, batch_size, hid_dim)
        返回:
            prediction (Tensor): 对当前时间步的词预测，形状为 (batch_size, output_dim)
            hidden (Tensor): 当前时间步的隐藏状态
            cell (Tensor): 当前时间步的细胞状态
        """
        # 将输入词转换为嵌入向量，添加维度以适配LSTM输入
        embedded = self.embedding(input.unsqueeze(0))  # embedded: (1, batch_size, emb_dim)
        # 通过LSTM层更新隐藏状态和细胞状态
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # 通过全连接层将隐藏状态转为词汇表大小的向量
        prediction = self.fc_out(output.squeeze(0))  # prediction: (batch_size, output_dim)
        return prediction, hidden, cell

#### 4. Seq2Seq模型的定义

In [4]:
# 定义 Seq2Seq 模型，将 Encoder 和 Decoder 结合
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        """
        初始化 Seq2Seq 模型。
        参数:
            encoder (Encoder): 编码器对象
            decoder (Decoder): 解码器对象
            device (torch.device): 指定运行设备（CPU 或 GPU）
        """
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        Seq2Seq 模型的前向传播。
        参数:
            src (Tensor): 编码器的输入张量，形状为 (src_len, batch_size)
            trg (Tensor): 解码器的目标输出张量，形状为 (trg_len, batch_size)
            teacher_forcing_ratio (float): 使用教师强制的概率
        返回:
            outputs (Tensor): 预测的输出序列张量，形状为 (trg_len, batch_size, output_dim)
        """
        # 获取目标序列长度和词汇表大小
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.fc_out.out_features

        # 初始化输出张量，用于存储每个时间步的预测结果
        outputs = torch.zeros(trg_len, trg.shape[1], trg_vocab_size).to(self.device)  
        # outputs: (trg_len, batch_size, output_dim)

        # 通过编码器获得隐藏状态和细胞状态
        hidden, cell = self.encoder(src)

        # 使用目标序列的第一个词作为解码器的初始输入
        input = trg[0]

        # 遍历目标序列的每个时间步
        for t in range(1, trg_len):
            # 获取当前时间步的预测、隐藏状态和细胞状态
            output, hidden, cell = self.decoder(input, hidden, cell)
            # 存储预测结果
            outputs[t] = output
            # 获取当前时间步的预测结果中概率最高的词
            top1 = output.argmax(dim=1)
            # 根据教师强制策略，决定解码器的下一个输入是目标词还是预测词
            input = trg[t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

#### 5. 训练和测试过程

In [5]:
!pip3 install fugashi
!pip3 install unidic_lite 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
from transformers import AutoTokenizer

# 加载日语分词器
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义模型参数
INPUT_DIM = len(tokenizer.vocab)
OUTPUT_DIM = len(tokenizer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512

# 初始化编码器、解码器和整体模型
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM).to(device)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

# 优化器和损失函数
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# 训练函数
def train(model, data, optimizer, criterion, clip=1):
    model.train()
    epoch_loss = 0
    for src, trg in data:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data)

# 推理函数
def generate_response(model, sentence, max_len=50):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    src = torch.tensor(token_ids).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src)
        input = src[0, :]

        outputs = []
        for _ in range(max_len):
            output, hidden, cell = model.decoder(input, hidden, cell)
            top1 = output.argmax(1)
            outputs.append(top1.item())
            input = top1
            if top1.item() == tokenizer.sep_token_id:
                break

    result = tokenizer.decode(outputs)
    return result

# 加载和预处理数据示例
def preprocess_data(dialogues):
    data = []
    for dialogue in dialogues:
        input_text = dialogue[0]
        target_text = dialogue[1]
        src = tokenizer.encode(input_text, add_special_tokens=True, max_length=128, truncation=True)
        trg = tokenizer.encode(target_text, add_special_tokens=True, max_length=128, truncation=True)

        src_tensor = torch.tensor(src).unsqueeze(1).to(device)  # (seq_len, 1)
        trg_tensor = torch.tensor(trg).unsqueeze(1).to(device)  # (seq_len, 1)
        # src_tensor = torch.tensor(src).to(device)
        # trg_tensor = torch.tensor(trg).to(device)
        data.append((src_tensor, trg_tensor))
    return data

# 示例数据
dialogues = [
    ("おはようございます。", "今日はとても良いお天気ですね。"),
    ("お昼ご飯に行きましょうか？", "ぜひ行きましょう。")
]
train_data = preprocess_data(dialogues)

# 训练模型
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_data, optimizer, criterion)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f}')

/home/vipuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1 Train Loss: 10.3914
Epoch 2 Train Loss: 10.1847
Epoch 3 Train Loss: 9.4286
Epoch 4 Train Loss: 8.5764
Epoch 5 Train Loss: 6.9589
Epoch 6 Train Loss: 5.1552
Epoch 7 Train Loss: 3.4129
Epoch 8 Train Loss: 2.1073
Epoch 9 Train Loss: 1.3875
Epoch 10 Train Loss: 1.0013


#### 6. 测试效果

In [7]:
# 测试生成
test_sentence = "おはようございます。"
print("生成的回答:", generate_response(model, test_sentence))

生成的回答: 今日 は とても 良い お 天気 です ね 。 [SEP]
